Just to check time of execution

In [2]:
%%time

import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import re

url_datos_madrid = 'https://datos.madrid.es/egob/catalogo/212769-0-atencion-medica.json'
response = requests.get(url_datos_madrid)

datos_json = response.json()

df_places = pd.json_normalize(datos_json['@graph'])
pd.set_option('display.max_colwidth',50)

df_bikes = pd.read_csv('../data/bicimad_stations.csv')


def lat(x):
    x1 = re.sub('[[]|[]]','',x)
    x2 = re.split('[,]\s',x1)
    return float(x2[1])

    
def long(x):
    x1 = re.sub('[[]|[]]','',x)
    x2 = re.split('[,]\s',x1)
    return float(x2[0])
    

df_bikes['Latitude'] = df_bikes.apply(lambda x: lat(x['geometry_coordinates']),axis=1)
df_bikes['Longitude'] = df_bikes.apply(lambda x: long(x['geometry_coordinates']),axis=1)

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

df_places['location_place'] = df_places.apply(lambda x: to_mercator(x['location.latitude'], x['location.longitude']),axis=1)
df_bikes['location_bikes'] = df_bikes.apply(lambda x: to_mercator(x['Latitude'], x['Longitude']),axis=1)

df_places_bikes = df_places.assign(key=0).merge(df_bikes.assign(key=0), how='left', on='key')

def distance_meters(loc_start, loc_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return loc_start.distance(loc_finish)

df_places_bikes['distance'] = df_places_bikes.apply(lambda x: distance_meters(x['location_place'], x['location_bikes']),axis=1)

def row_filter(df, cat_var, cat_values):
    df = df[df[cat_var].isin(cat_values)]
    return df.reset_index(drop=True)

place_selected = ['Centro Concertado de Atención a las Adicciones (CCAD) Cáritas Madrid']

result = row_filter(df_places_bikes,'title',place_selected).sort_values(by='distance', ascending=True)[['address','distance']]

result.head()

CPU times: user 32.6 s, sys: 2.37 s, total: 34.9 s
Wall time: 36.7 s


,address,distance
252,Calle Santa Hortensia nº 31,435.795195
198,Calle Suero de Quiñones nº 2,582.740624
197,Calle Padre Indalecio Hernández nº 1,954.237158
243,Avenida de Camilo José Cela nº 23,959.712500
149,Calle María Francisca nº 1,963.232462


In [21]:
print('La estación BiciMAD más cercana está en: ', result.iloc[0]['address'], '. Se encuentra a ', round(result.iloc[0]['distance'],1), 'metros', '\n', 'La siguiente se encuentra en ', result.iloc[1]['address'], 'a ', round(result.iloc[1]['distance'],1), 'metros')

La estación BiciMAD más cercana está en:  Calle Santa Hortensia nº 31 . Se encuentra a  435.8 metros 
 La siguiente se encuentra en  Calle Suero de Quiñones nº 2 a  582.7 metros
